Summarization engine videos on YouTube:
- https://www.youtube.com/watch?v=ZhVAjVraiRQ

In [ ]:
# Libraries for our code
import os
import re
import pandas as pd
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import torch


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
!pip install contractions

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 6.3 MB/s eta 0:00:00


In [ ]:
# DO NOT RUN: This is used to connect Google Drive to add the files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def remove_footer(text):
  """
  Removes the footer using a combined regex pattern with  the lookahead assertion.

  Args:
      text: The text content.

  Returns:
      Text without the footer.
  """
  combined_pattern = r"""
    (  # Start of group
      # ... (previous patterns for cookies, copyright, etc.) ...

      |  # OR
      About\s*Us\s*\|?\s*Contact\s*Us\s*\|?\s*FAQ\s*\|?\s*Help  # Common page links
      |  # OR
      Terms\s*and\s*Conditions\s*\|?\s*Privacy\s*Statement  # Legal links
      |  # OR
      Cookie\s*Policy\s*\|?\s*Accessibility\s*\|?\s*Sitemap  # More links
      |  # OR
      All\s*Rights\s*Reserved\s*\|?\s*Copyright\s*\d{4}  # Copyright variations
      |  # OR
      [a-zA-Z\s]+\s*is\s*a\s*(registered\s*)?trademark\s*of\s*[a-zA-Z\s]+\.  # Trademark variations
      |  # OR
      Get\s*in\s*touch\s*\|?\s*Send\s*us\s*a\s*message  # Contact prompts
      |  # OR
      Back\s*to\s*top\s*\|?\s*Top  # Back to top links
      |  # OR
      You\s*are\s*here:\s*[\w\s\/\>]+  # Breadcrumb trails
      |  # OR
      (View\s*|Edit)\s*this\s*page\s*on\s*[\w\s\.]+  # Edit/view links
      |  # OR
      Last\s*updated:\s*\d{1,2}\s*[a-zA-Z]+\s*\d{4}  # Last updated info
      |  # OR
      Report\s*a\s*problem\s*\|?\s*Suggest\s*an\s*edit  # Feedback options
      |  # OR
      Disclaimer:\s*.*  # Disclaimers
      |  # OR
      [a-zA-Z\s]+\s*Sitemap  # Sitemap variations
      |  # OR
      [a-zA-Z\s]+\s*Terms\s*and\s*Conditions  # Terms and conditions variations
      |  # OR
      [a-zA-Z\s]+\s*Privacy\s*Policy  # Privacy policy variations
      |  # OR
      [a-zA-Z\s]+\s*Contact\s*Us  # Contact us variations
      |  # OR
      [a-zA-Z\s]+\s*FAQ  # FAQ variations
      |  # OR
      [a-zA-Z\s]+\s*Help  # Help variations
      |  # OR
      [a-zA-Z\s]+\s*About\s*Us  # About us variations
      |  # OR
      [a-zA-Z\s]+\s*About\s*Author  # Author variations
      |  # OR
      [a-zA-Z\s]+\s*Conditions  # Conditions variations
      |  # OR
      (To\s*read\s*more\s*on\s*related\s*topics|Enjoy\s*extended\s*coverage).*?  # Read more/extended coverage
      |  # OR
      (access\s*subscription|Subscribe\s*to\s*our\s*newsletter|Make\s*a\s*donation).*?  # Subscription/donation prompts
      |  # OR
      (Advertise\s*with\s*us|Advertising\s*options|Become\s*a\s*sponsor).*?  # Advertising prompts
      |  # OR
      Design,\s*CMS,\s*Hosting\s*&\s*Web\s*Development  # Website development info
      |  # OR
      (Subscribe\s*now|Sign\s*up\s*for\s*a\s*(free|premium)\s*subscription|Upgrade\s*to\s*premium|Get\s*(full|unlimited)\s*access).*?  # Subscription variations


      # We can add more footer patterns here with "|" (OR)

    )  # End of group
    (?=\s*$)  # Positive lookahead assertion for end of string
  """
  cleaned_text = re.sub(combined_pattern, '', text, flags=re.IGNORECASE | re.DOTALL | re.VERBOSE)
  return cleaned_text

In [ ]:

# Function to extract the main content from HTML
def extract_main_content(html_text):
    """
    Extracts the main content from HTML using BeautifulSoup.
    """
    soup = BeautifulSoup(html_text, 'html.parser')

    # Attempting to extract main content by analyzing HTML tags
    for selector in ['article', 'main', 'content', 'post']:
        content = soup.find_all(selector)
        if content:
            return ' '.join([p.get_text() for p in content])

    # Fallback: Extract text from paragraphs if specific tags are not found
    return ' '.join([p.get_text() for p in soup.find_all('p')])

In [ ]:
# Path to the shared folder, scraped from Timo
folder_path = '/content/drive/MyDrive/BTT: Accenture 1C/scraped'
# Path to the new folder with the corrected data
dest_path = '/content/drive/MyDrive/BTT: Accenture 1C/cleaned'


# Feel free to uncomment to view the contents of the folder
#os.listdir(folder_path)

In [ ]:
def filter_by_keywords(content, threshold):
    """
    Filters articles based on tech-related keywords.

    Args:
        content: The text content of the article.
        keywords: A list of tech-related keywords.
        threshold: Minimum number of keyword matches to keep the article.

    Returns:
        Boolean indicating whether the article meets the keyword threshold.
    """
    # print('Filtering by keywords...')

    keywords = ["technology", "ai", "software", "biotech", "startup", "tech", "machine learning","investment", "engineering", "fintech", "artificial intelligence", "silicon valley", "financial", "infrastructure", "blockchain"]

    keyword_count = sum([content.count(keyword) for keyword in keywords])
    return keyword_count >= threshold


In [ ]:


def remove_irrelevant_sections(text):
    """
    Removes irrelevant sections from the article text.
    """
    # Define irrelevant patterns
    irrelevant_patterns = [
        r'copyright.*', r'cookie settings.*', r'donation.*', r'subscribe.*',
        r'©.*', r'privacy policy.*', r'forgot username.*', r'forgot password.*',
        r'user agreement.*', r'contact us.*', r'earn commission.*', r'affiliate link.*',
        r'click.*', r'subscribe.*', r'visit.*', r'read.*', r'article.*'
    ]

    # Remove irrelevant content based on patterns
    for pattern in irrelevant_patterns:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE | re.DOTALL)

    # Clean up any residual irrelevant text
    text = re.sub(r'\n+', '\n', text)  # Remove excessive newlines
    return text.strip()


In [ ]:
def remove_error_messages(text):
    # print('Removing error messages...')
    error_patterns = [
    r'404 not found', r'403 forbidden', r'500 internal server error',
    r'502 Bad Gateway', r'503 Service Unavailable', r'504 Gateway Timeout',
    r'400 Bad Request', r'401 Unauthorized', r'408 Request Timeout',
    r'Page not found', r'Access denied', r'Try again later',
    r'This content is available to subscribers only', r'Sign in to read more',
    r'Your connection is not private', r'Under maintenance',
    r'Connection refused', r'Oops, something went wrong'
]

    for pattern in error_patterns:
        if re.search(pattern, text):
            return None  # Skip this article
    return text

In [ ]:
def filter_short_articles(text, min_word_count=150):
    # print('Removing short articles...')
    words = word_tokenize(text)
    return text if len(words) >= min_word_count else None

In [ ]:

def tokenize_text(text):
    """
    Tokenizes the provided text into words.

    Args:
        text: The text to be tokenized.

    Returns:
        List of tokenized words.
    """
    # print("Tokenizing text...")
    # Tokenize the text
    tokens = word_tokenize(text)
    return tokens

In [ ]:
# Contractions (e.g., "don't", "isn't") will be expanded to their full forms for consistency before applying lemmatization
import contractions

def expand_contractions(text):
    return contractions.fix(text)


In [ ]:

def lemmatize_tokens(tokens):
    """
    Lemmatizes the provided tokens using WordNetLemmatizer.

    Args:
        tokens: List of tokenized words.

    Returns:
        List of lemmatized words.
    """
    # print("Lemmatizing tokens...")
    # tokens = expand_contractions(tokens)
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in tokens]

In [ ]:
def normalize_text(text):
    # print("Normalizing text...")
    text = text.lower()
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return filtered_tokens

In [ ]:
def clean_article(article_path):
    # print(f"Cleaning article: {article_path}")
    with open(article_path, 'r') as file:
        text = file.read()

    if not text:
        print(1)
        return None


    text = expand_contractions(text)
    text = text.lower()

    if not filter_short_articles(text):
        print(2)
        return None

    if filter_by_keywords(text, 1) is None:
        print(3)
        return None

    text = remove_error_messages(text)
    if text is None:
        print(4)
        return None

    text = remove_footer(text)


    # Apply tokenization and normalization
    tokens = tokenize_text(text)
    normalized_tokens = normalize_text(' '.join(tokens))
    lemmatized_tokens = lemmatize_tokens(normalized_tokens)
    return ' '.join(lemmatized_tokens)

In [ ]:
def process_files(input_folder, dest_folder):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    cleaned_data = []
    rejected_data = []
    count = 1

    for filename in os.listdir(input_folder):
        if count > 50:
            break  # Only process the first 50 files

        if not filename.endswith('.txt'):
            continue  # Skip non-text files

        file_path = os.path.join(input_folder, filename)

        # Read the content of the file
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()

        # Clean the article
        cleaned_text = clean_article(file_path)
        if cleaned_text:
            cleaned_data.append([filename, cleaned_text, content])
        else:
            # If cleaning returns None, log the rejection
            rejected_data.append([filename, content, "Rejected"])

        count += 1

    # Save cleaned data to a CSV
    df_cleaned = pd.DataFrame(cleaned_data, columns=['Filename', 'Cleaned Text', 'Original Text'])
    df_cleaned.to_csv(os.path.join(dest_folder, 'cleaned_articles.csv'), index=False)

    # Save rejected files to another CSV
    df_rejected = pd.DataFrame(rejected_data, columns=['Filename', 'Content', 'Reason'])
    df_rejected.to_csv(os.path.join(dest_folder, 'rejected_articles.csv'), index=False)

In [ ]:
input_folder = '/content/drive/MyDrive/BTT: Accenture 1c/scraped'
dest_folder = '/content/drive/MyDrive/BTT: Accenture 1C/cleaned'
process_files(input_folder, dest_folder)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/BTT: Accenture 1c/scraped'

In [ ]:
# Extracting entities to identify key players, companies, products, or organizations in the industry
def extract_named_entities(text):
    doc = nlp(text)
    entities = [(entity.text, entity.label_) for entity in doc.ents]
    return entities

# Apply to the DataFrame


In [ ]:
#checking URLS for tech domains and tech words. if you want it to just look for domains, delete tech
tech_sources = ['gizmodo.com','cnn.com/tech','cnet.com','tech'] #common tech domains and tech words
def is_tech_source(url):
  return any(source in url for source in tech_sources)
example_url = 'https://www.cnn.com/business/tech'
result = is_tech_source(example_url)
print(result)

True


In [ ]:
testcase_1 = "The Week 17 NFL schedule opens with Malik Willis and the Tennessee Titans hosting Dak Prescott and the Dallas Cowboys on Thursday Night Football. All-Pro running back Derrick Henry (hip) is doubtful and Josh Dobbs is expected to make his first start of the season in place of Ryan Tannehill and Malik Willis for Tennessee. Before locking in any Titans vs. Cowboys Showdown NFL DFS lineups for Thursday Night Football on sites like DraftKings and FanDuel, you NEED to see what SportsLine's Jimmie Kaylor has to say. Kaylor is a NFL and DFS expert for SportsLine, who opened the 2022 NFL season by winning DraftKings' NFL Showdown Thursday Kickoff Millionaire contest. He edged out over 355,000 other entrants in the contest and also has multiple five-figure tournament wins on his DFS resume. Kaylor, who has been cashing big all season, has covered the NFL and college football for close to a decade as a member of the Pro Football Writers of America, and his background as a former college and NFL player gives him a unique perspective when building his fantasy lineups and locking in his betting picks.   Kaylor has been spot-on all season, hitting on epic NFL DFS picks like Jerry Jeudy (4-102-1), Stefon Diggs (12-148-3, 6-106-1, 7-92-1), Amari Cooper (7-101-1, 5-131-1), Nick Chubb (23-113-1, 23-101-2), CeeDee Lamb (8-87-1), Deebo Samuel (6-115-1) Cooper Kupp (14-122), Travis Kelce (7-25-4, 10-106), Davante Adams (3-124-2), Justin Fields (179-1-1, 82-1), Derrick Henry (132-2), Jimmy Garoppolo (228-4), Michael Pittman (7-61-1), Christian McCaffrey (26-108-1, 6-30), Kirk Cousins (460-4), Justin Jefferson (12-123-1), J.K. Dobbins (13-125), Aaron Jones (17-90, 4-36-1), and T.J. Hockenson (13-109-2). Anyone who has followed his picks is WAY UP! Now, Kaylor has turned his attention to Cowboys vs. Titans on Week 17 Thursday Night Football and locked in his top daily Fantasy football picks. We can tell you that one of Kaylor's top NFL DFS picks for the Titans vs. Cowboys TNF matchup is Dallas wide receiver CeeDee Lamb. 'On paper this matchup looks outstanding for CeeDee Lamb and the Dallas passing attack. Tennessee has one of the best rushing defenses in the NFL, but has been highly susceptible against the pass. Lamb has been on a tear, hauling in 17 passes for 246 yards and two touchdowns in his last two games. My biggest concern is that Dallas will jump out to a big early lead with Tennessee being decimated by injuries. Still, I expect Lamb to be active early and notch his third consecutive 100-yard receiving performance.' Kaylor is also targeting another under-the-radar option for TNF who could post HUGE numbers and carry your lineups to victory on Thursday Night Football. This pick could be the difference between winning your tournaments and cash games or going home with nothing! You ABSOLUTELY need to see who it is before locking in any lineups.  Be sure to follow Jimmie (@jimmiekaylor) on Twitter for notifications about rosters changes/updates.   What are the top NFL DFS picks for Titans vs. Cowboys on FanDuel and DraftKings? And which player is a MUST-ROSTER? ... Join SportsLine now to see NFL expert and DraftKings Millionaire winner Jimmie Kaylor's top picks, stacks, and player pools for FanDuel and DraftKings, and cash in BIG on NFL DFS! Share This Story  2024 Detroit Lions futures picks: Breaking down win totals, Super Bowl odds, schedule, depth chart and more Florida State vs. Georgia Tech prediction, odds, spread, line, start time: Proven expert releases college football picks, best bets, props for College Football Classic 2024 MLB pitching props: Robbie Ray among expert's best bets for Tuesday, August 20 2024 Chicago Bears futures picks: Breaking down win totals, Super Bowl odds, schedule, depth chart and more"
clean_article('/content/drive/MyDrive/BTT: Accenture 1C/scraped/5454832.txt')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/BTT: Accenture 1C/scraped/5454832.txt'